3168个录制的声音样本（来自男性和女性演讲者），采集的频率范围是0hz-280hz，已经对数据进行了预处理; 一共有21个属性值，请判断该声音是男还是女？
使用Accuracy作为评价标准

#### 思路
1. 加载数据 & 数据处理
2. 切分训练集及测试集
3. 使用不同模型, 计算accuracy, 看效果 

In [34]:
import pandas as pd
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')


In [56]:
#计算accuracy 并将每次的结果存到df

def calc_accuracy(y_predict, y_test, model_name, score_all):
    
    score_each = pd.Series()

    y_predict = [1 if x>=0.5 else 0 for x in y_predict] #将预测概率转化成0,1
    acc_score = accuracy_score(y_predict, y_test) #计算accuracy
    

    score_each['model'] = model_name
    score_each['score'] = acc_score
    score_each = score_each.to_frame().T
    score_all = pd.concat([score_all,score_each], axis= 0)

    print('Accuracy Score: %0.4f' % acc_score)
    return score_all


In [47]:
#加载数据
voice = pd.read_csv(r'E:\bi_course\L6\tue_course\课堂资料\L6-code\code\tue_course\voice\voice.csv')
print('数据量:',voice.shape[0],'数据维度:',voice.shape[1])
# print(voice.isna().sum()) #查看是否有空值
voice.head(5)


数据量: 3168 数据维度: 21


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [48]:
#用labelEncoder将label改成0,1
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
voice['label'] = le.fit_transform(voice['label']) #male 是1, female是0
# print(voice)

X = voice.iloc[:,:-1] #去除label列
y =  voice.iloc[:,-1] #只保留label列

attr_ls = list(X.columns) #特征属性list

#切分train, test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
score_all = pd.DataFrame() #初始化

#用lightgbm训练 并输出accuracy分数

import lightgbm as lgb
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', 
                         'metric': 'auc', #评估指标 
                         'learning_rate' : 0.01, #学习率
                         'max_depth' : 20, #树的最大深度
                         'feature_fraction':0.8, #每次迭代中特征使用比例
                         'bagging_fraction': 0.9, #样本采样比例
                         'bagging_freq': 8, #bagging次数
                         'lambda_l1': 0.6, #L1正则
                         'lambda_l2': 0 #  越小l2正则程度越高  
        }


#转化成lgb的数据结构
train_data = lgb.Dataset(X_train, label=y_train) 
test_data = lgb.Dataset(X_test, label=y_test) 

#参数输入到lgb模型中
model = lgb.train(param,
                    train_data,
                    valid_sets= [test_data,train_data],
                    num_boost_round = 1000,
                    early_stopping_rounds=200,
                    verbose_eval=25
                    
                    )

#预测
y_predict = model.predict(X_test) 
#输出accuracy分数
score_all = calc_accuracy(y_predict, y_test,'lgb',score_all)

Training until validation scores don&#39;t improve for 200 rounds
[25]	training&#39;s auc: 0.99808	valid_0&#39;s auc: 0.995514
[50]	training&#39;s auc: 0.998244	valid_0&#39;s auc: 0.995099
[75]	training&#39;s auc: 0.998258	valid_0&#39;s auc: 0.995304
[100]	training&#39;s auc: 0.998416	valid_0&#39;s auc: 0.995774
[125]	training&#39;s auc: 0.998572	valid_0&#39;s auc: 0.995684
[150]	training&#39;s auc: 0.99878	valid_0&#39;s auc: 0.995894
[175]	training&#39;s auc: 0.998981	valid_0&#39;s auc: 0.995984
[200]	training&#39;s auc: 0.999159	valid_0&#39;s auc: 0.996203
[225]	training&#39;s auc: 0.999335	valid_0&#39;s auc: 0.996123
[250]	training&#39;s auc: 0.999483	valid_0&#39;s auc: 0.996103
[275]	training&#39;s auc: 0.999585	valid_0&#39;s auc: 0.996403
[300]	training&#39;s auc: 0.99968	valid_0&#39;s auc: 0.996933
[325]	training&#39;s auc: 0.999755	valid_0&#39;s auc: 0.997202
[350]	training&#39;s auc: 0.999809	valid_0&#39;s auc: 0.997272
[375]	training&#39;s auc: 0.999865	valid_0&#39;s auc: 0.99

In [71]:
#用xgboost训练 并输出accuracy分数

import xgboost as xgb
param = {'boosting_type':'gbdt',
        'objective' : 'binary:logistic', #
        'eval_metric' : 'auc',
        'eta' : 0.01,
        'max_depth' : 20,
        'colsample_bytree':0.8,
        'subsample': 0.9,
        'subsample_freq': 8,
        'alpha': 0.6,
        'lambda': 0,
        }

#转化成xgboost数据格式
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)

#训练
model = xgb.train(param, 
    train_data, 
    evals=[(train_data, 'train'), (test_data, 'test')], 
    num_boost_round = 10000, 
    early_stopping_rounds=200, 
    verbose_eval=25)


#预测
y_predict = model.predict(test_data) 

#输出accuracy分数
score_all = calc_accuracy(y_predict, y_test,'xgb',score_all)

[0]	train-auc:0.995294	test-auc:0.986582
Multiple eval metrics have been passed: &#39;test-auc&#39; will be used for early stopping.

Will train until test-auc hasn&#39;t improved in 200 rounds.
[25]	train-auc:0.99936	test-auc:0.995579
[50]	train-auc:0.999584	test-auc:0.995324
[75]	train-auc:0.999632	test-auc:0.995329
[100]	train-auc:0.999713	test-auc:0.995499
[125]	train-auc:0.99976	test-auc:0.995644
[150]	train-auc:0.999812	test-auc:0.995914
[175]	train-auc:0.999865	test-auc:0.996123
[200]	train-auc:0.9999	test-auc:0.996363
[225]	train-auc:0.999925	test-auc:0.996543
[250]	train-auc:0.999945	test-auc:0.996643
[275]	train-auc:0.999957	test-auc:0.996683
[300]	train-auc:0.999969	test-auc:0.996913
[325]	train-auc:0.999978	test-auc:0.997332
[350]	train-auc:0.999981	test-auc:0.997352
[375]	train-auc:0.999989	test-auc:0.997402
[400]	train-auc:0.999992	test-auc:0.997492
[425]	train-auc:0.999996	test-auc:0.997422
[450]	train-auc:0.999996	test-auc:0.997382
[475]	train-auc:0.999996	test-auc:0.99

In [72]:
#调用sklearn的GBDT模型
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state=10)

#训练
model.fit(X_train, y_train)

#预测
y_predict = model.predict(X_test) 
#输出accuracy分数
score_all = calc_accuracy(y_predict, y_test,'GBDT',score_all)

Accuracy Score: 0.9732


In [73]:
#调用sklearn的LR模型
from sklearn.linear_model.logistic import LogisticRegression

model = LogisticRegression(max_iter=1000, 
                           verbose=True, 
                           random_state=33
                          )

model.fit(X_train, y_train)

#预测
y_predict = model.predict(X_test) 
#输出accuracy分数
score_all = calc_accuracy(y_predict, y_test,'LR',score_all)

Accuracy Score: 0.9842
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [74]:
#调用sklearn的SVC模型
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import MinMaxScaler

#数据归一化
scale = MinMaxScaler(feature_range=(0, 1))
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

model = LinearSVC(
			max_iter=1000,
            verbose=True,
			random_state=33
			
		   )

model.fit(X_train, y_train)

#预测
y_predict = model.predict(X_test) 
#输出accuracy分数
score_all = calc_accuracy(y_predict, y_test,'SVC',score_all)

[LibLinear]Accuracy Score: 0.9826


In [75]:
score_all.sort_values(by = 'score', ascending = False)

,model,score
0,LR,0.984227
0,SVC,0.98265
0,lgb,0.979495
0,xgb,0.977918
0,xgb,0.977918
0,GBDT,0.973186


### thinking conclusion
- 使用5个模型预测男女声音, 得到最高accuracy score是LR模型
- 感觉分数都太高(尤其是跑auc值时), 有过拟合嫌疑? 看老师案例, 除了X_train, X_test数据, lgb和xgb 都使用了另一份数据test(样本外数据)